In [1]:
import pickle
import dotenv

In [2]:
import tqdm
import time
import re

In [3]:
from mmw.parse import *

In [4]:
dotenv.load_dotenv()

True

In [5]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

'\n\nThis is indeed a test'

In [6]:
questions_explanation = parse_question_explanations()

In [7]:
all_parsed_essays = parse_essays()

In [8]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [9]:
len(all_parsed_essays), len(filtered_marked_essays)

(77, 76)

In [10]:
#model = "text-davinci-003"
#model = "text-ada-001"
model = "gpt-3.5-turbo"

#models simple/ cheap/ fast to advanced:
#model = "ada"
#model = "babbage"
#model = "curie"
#model = "davinci"

#model = "ada:ft-mark-my-words-2023-05-01-11-44-26"

In [11]:
prompt_tpl = 'You are a caring teacher who wants to help their students succeed by understanding their mistakes and how to improve. Given is a student essay and a question to a skill. Give a proper remark and detailed explanation to grade the skill, along with references from the Essay. Use direct form to answer the student. Start with a positive feedback. Afterwards grade the student. Grades can be numbers between 0 (lowest, worst) and 3 (highest, best).\n\n###\n\n'
prompt_tpl += 'Essay:\n{essay}\n\n###\n\n'
prompt_tpl += 'Question:\n{explanation}\n{question}\n\n###\n\n'
#prompt_tpl += 'Grade: {remark}\n\n###\n\n'
prompt_tpl += 'Answer:'

In [12]:
def assemble_citation_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation):

    all_prompts = []
    for i,essay_array in tqdm.tqdm(enumerate(all_parsed_essays)):

        essay_id = int(re.findall(r'\d+', essay_array[0])[-1])
        cur_row = filtered_marked_essays[filtered_marked_essays['essay id'] == essay_id]
        if len(cur_row) != 1:
            continue

        for j,k in questions_explanation.iterrows():
            question = k.question
            explanation = k.explanation
            grade = int(cur_row.iloc[0,j+1])
            prompt = prompt_tpl.format(essay=essay_array[3][:-1], #implicit newline
                                    question=question.strip(),
                                    explanation=explanation.strip(),
                                    remark=grade)

            all_prompts.append((i, j, prompt, grade))
    return all_prompts


In [13]:
all_prompts = assemble_citation_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation)

77it [00:00, 368.55it/s]


In [18]:
all_answers = []
for prompt_array in tqdm.tqdm(all_prompts): #[:LIMIT_PROMPTS*len(questions_explanation)]):
    for i in range(3):
        try:
            i, j, prompt, annot = prompt_array
            all_splits = prompt.split('\n\n###\n\n')
            message = '\n\n###\n\n'.join(all_splits[1:])
            response = openai.ChatCompletion.create(model=model, messages=[
                {'role': 'system', 'content': all_splits[0]},
                {'role': 'user', 'content': message}],
                                                    temperature=0, max_tokens=250)
            all_answers.append((i, j, prompt, response, annot))
            
            time.sleep(.5)
            break
        except:
            print(f"exception, retrying {i}")
            time.sleep(4.)

  4%|▍         | 39/988 [10:43<4:49:22, 18.30s/it]

exception, retrying 3


  9%|▉         | 93/988 [26:31<3:52:22, 15.58s/it]

exception, retrying 7


 11%|█▏        | 112/988 [32:39<3:53:28, 15.99s/it]

...

 60%|██████    | 596/988 [23:00:50<1:44:13, 15.95s/it]  

exception, retrying 45


 97%|█████████▋| 961/988 [24:13:08<06:11, 13.77s/it]  

exception, retrying 74


100%|██████████| 988/988 [24:19:01<00:00, 88.60s/it]


In [15]:
all_answers[0]

(0,
 0,
 "You are a caring teacher who wants to help their students succeed by understanding their mistakes and how to improve. Given is a student essay and a question to a skill. Give a proper remark and detailed explanation to grade the skill, along with references from the Essay. Use direct form to answer the student. Start with a positive feedback. Afterwards grade the student. Grades can be numbers between 0 (lowest, worst) and 3 (highest, best).\n\n###\n\nEssay:\nThe Cage\nIt felt like I was walking forever, i didn't even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is ra

In [19]:
save_results('data/evaluations/run_10_chatgpt_both', all_parsed_essays, all_questions, all_answers)

In [17]:
all_answers[0][3]['choices'][0]['message']['content']

"Great job on starting your essay with an interesting and attention-grabbing opening! You began with a description of a creepy alleyway and the feeling of walking forever, which immediately sets the tone for the rest of the narrative. Additionally, your use of thunder and rain added a sense of urgency and danger to the scene. \n\nHowever, there were a few areas where your opening could be improved. Firstly, there were some spelling and grammatical errors that could be corrected with proofreading. Secondly, the transition from the alleyway to school felt a bit abrupt and disconnected. It would have been helpful to have a clearer connection between the two settings. \n\nOverall, I would grade your skill in writing a compelling opening as a 2 out of 3. You did a great job of setting the scene and grabbing the reader's attention, but there were some areas where the writing could be improved. Keep up the good work!"